In [11]:
import os
import re
from functools import reduce,partial
from glob import glob
from collections import Counter

import pandas as pd
import dill

In [12]:
with open('./read_csv.pkl', 'rb') as f:
    read_csv = dill.load(f)

with open('./preproc_cols.pkl', 'rb') as f:
    preproc_cols = dill.load(f)

In [13]:
def get_file_date(file_path):
    file_name = os.path.basename(file_path)
    date = re.findall(r"\d{8}|\d{4}", file_name)[-1]
    if len(date) == 4:
        return date + "1231"
    elif len(date) == 8:
        return date
    else:
        raise ValueError

In [14]:
# RawFiles
file_paths = glob("./data/raw_data/*.csv")
file_paths

['./data/raw_data/경찰청_범죄발생지_2012.csv',
 './data/raw_data/경찰청_범죄발생지_2013.csv',
 './data/raw_data/경찰청_범죄 발생 지역별 통계_20191231.csv',
 './data/raw_data/경찰청_범죄발생지_2014.csv',
 './data/raw_data/경찰청_범죄 발생 지역별 통계_20181231.csv',
 './data/raw_data/경찰청_범죄발생지_2017.csv',
 './data/raw_data/경찰청_범죄발생지_2016.csv',
 './data/raw_data/경찰청_범죄 발생 지역별 통계_20221231.csv',
 './data/raw_data/경찰청_범죄 발생 지역별 통계_20211231.csv',
 './data/raw_data/경찰청_범죄발생지_20151231.csv',
 './data/raw_data/경찰청_범죄 발생 지역별 통계_20201231.csv']

In [15]:
file_column_type_dict = dict()
df_list = list()
for file_path in file_paths:
    df = read_csv(file_path)
    df.columns = [preproc_cols(col) for col in df.columns]
    df['time'] = get_file_date(file_path)
    df_list.append(df)

In [16]:
df = pd.concat(df_list, axis=0)

In [18]:
melt_df = df.melt(id_vars=['범죄대분류', '범죄중분류', 'time'])

In [19]:
melt_df.isna().any()

범죄대분류        True
범죄중분류        True
time        False
variable    False
value        True
dtype: bool

In [20]:
melt_df[melt_df['범죄대분류'].isna()]

,범죄대분류,범죄중분류,time,variable,value
187,NaN,NaN,20181231,서울,NaN
188,NaN,NaN,20181231,서울,NaN
189,NaN,NaN,20181231,서울,NaN
190,NaN,NaN,20181231,서울,NaN
606,NaN,NaN,20181231,부산,NaN
...,...,...,...,...,...
36224,NaN,NaN,20181231,경남 통영,NaN
36640,NaN,NaN,20181231,제주 서귀포,NaN
36641,NaN,NaN,20181231,제주 서귀포,NaN
36642,NaN,NaN,20181231,제주 서귀포,NaN


In [21]:
melt_df = melt_df.dropna(subset=['범죄대분류'])

In [22]:
melt_df[melt_df['범죄중분류'].isna()]

,범죄대분류,범죄중분류,time,variable,value


In [23]:
melt_df[melt_df['variable'].isna()]

,범죄대분류,범죄중분류,time,variable,value


In [24]:
melt_df[melt_df['value'].isna()]

,범죄대분류,범죄중분류,time,variable,value
5520,강력범죄,살인기수,20191231,경남 마산,NaN
5521,강력범죄,살인미수등,20191231,경남 마산,NaN
5522,강력범죄,강도,20191231,경남 마산,NaN
5523,강력범죄,강간,20191231,경남 마산,NaN
5524,강력범죄,유사강간,20191231,경남 마산,NaN
...,...,...,...,...,...
36597,노동범죄,노동범죄,20141231,제주 서귀포,NaN
36598,안보범죄,안보범죄,20141231,제주 서귀포,NaN
36599,선거범죄,선거범죄,20141231,제주 서귀포,NaN
36600,병역범죄,병역범죄,20141231,제주 서귀포,NaN


In [25]:
melt_df['value'] = melt_df['value'].fillna(0).astype(int)

In [26]:
melt_df.sort_values("범죄대분류")

,범죄대분류,범죄중분류,time,variable,value
0,강력범죄,살인기수,20121231,서울,66
13717,강력범죄,유사강간,20211231,경남 김해,8
13716,강력범죄,강간,20211231,경남 김해,40
13715,강력범죄,강도,20211231,경남 김해,8
13714,강력범죄,살인미수등,20211231,경남 김해,2
...,...,...,...,...,...
13969,환경범죄,환경범죄,20141231,전남 순천,9
1818,환경범죄,환경범죄,20141231,광주,11
25895,환경범죄,환경범죄,20211231,경기 양주,80
35532,환경범죄,환경범죄,20211231,경남 밀양,52


In [27]:
melt_df.sort_values("범죄중분류")

,범죄대분류,범죄중분류,time,variable,value
10627,강력범죄,강간,20181231,경북 구미,40
30739,강력범죄,강간,20181231,충남 공주,15
6098,강력범죄,강간,20161231,경남 창원,68
11770,강력범죄,강간,20131231,강원 춘천,27
22778,강력범죄,강간,20181231,도시이외,353
...,...,...,...,...,...
13505,지능범죄,횡령,20191231,경남 김해,513
27602,지능범죄,횡령,20151231,경기 포천,132
15600,지능범죄,횡령,20191231,충북 충주,160
22799,지능범죄,횡령,20181231,도시이외,1926


In [28]:
melt_df.sort_values("variable")

,범죄대분류,범죄중분류,time,variable,value
15502,기타범죄,기타범죄,20201231,강원 강릉,782
15215,지능범죄,통화,20141231,강원 강릉,11
15216,지능범죄,문서인장,20141231,강원 강릉,68
15217,지능범죄,유가증권인지,20141231,강원 강릉,0
15218,지능범죄,사기,20141231,강원 강릉,926
...,...,...,...,...,...
15787,지능범죄,직무유기,20221231,충북 충주,4
15788,지능범죄,직권남용,20221231,충북 충주,12
15789,지능범죄,증수뢰,20221231,충북 충주,1
15713,지능범죄,증수뢰,20171231,충북 충주,2


In [29]:
melt_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 36520 entries, 0 to 36871
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   범죄대분류     36520 non-null  object
 1   범죄중분류     36520 non-null  object
 2   time      36520 non-null  object
 3   variable  36520 non-null  object
 4   value     36520 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 1.7+ MB


In [30]:
object_columns = melt_df.select_dtypes(object).columns
melt_df.loc[:, object_columns] = melt_df.loc[:, object_columns].map(lambda x : x.strip())

In [31]:
melt_df.convert_dtypes().info()

<class 'pandas.core.frame.DataFrame'>
Index: 36520 entries, 0 to 36871
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   범죄대분류     36520 non-null  string
 1   범죄중분류     36520 non-null  string
 2   time      36520 non-null  string
 3   variable  36520 non-null  string
 4   value     36520 non-null  Int64 
dtypes: Int64(1), string(4)
memory usage: 1.7 MB


In [32]:
melt_df=melt_df.convert_dtypes()

In [33]:
melt_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 36520 entries, 0 to 36871
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   범죄대분류     36520 non-null  string
 1   범죄중분류     36520 non-null  string
 2   time      36520 non-null  string
 3   variable  36520 non-null  string
 4   value     36520 non-null  Int64 
dtypes: Int64(1), string(4)
memory usage: 1.7 MB


In [34]:
set(melt_df['범죄대분류'])

{'강력범죄',
 '교통범죄',
 '기타범죄',
 '노동범죄',
 '마약범죄',
 '병역범죄',
 '보건범죄',
 '선거범죄',
 '안보범죄',
 '절도범죄',
 '지능범죄',
 '특별경제범죄',
 '폭력범죄',
 '풍속범죄',
 '환경범죄'}

In [37]:
melt_df.reset_index(drop=True).to_csv("./data/data.csv", index=False)